In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from app.pipelines import ingest_sample_documents
ingest_sample_documents()

In [3]:
from app.utils import gpt_41

gpt_41.invoke("Who is the most known Konrad Bujak in Poland in Product Management")

NotFoundError: Error code: 404 - {'requestId': '3486618d-c825-40ed-8124-b85b22208a50', 'statusCode': 404, 'timestamp': '2025-05-06T13:48:32.260Z', 'path': '/v1/responses', 'message': 'Cannot POST /v1/responses'}